In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(1)

In [4]:
cd C:\Users\Marcus\Desktop\python\alf_sports_betting_predictor\data

C:\Users\Marcus\Desktop\python\alf_sports_betting_predictor\data


In [5]:
df = pd.read_csv('20210317 AFL_2020_stats_win_loss_normalised.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   kicks_home_team                     153 non-null    float64
 1   one-percenters_home_team            153 non-null    float64
 2   kicks_away_team                     153 non-null    float64
 3   one-percenters_away_team            153 non-null    float64
 4   home_win_lose                       153 non-null    int64  
 5   away_win_lose                       153 non-null    int64  
 6   kicks_home_team_normalise           153 non-null    float64
 7   one-percenters_home_team_normalise  153 non-null    float64
 8   kicks_away_team_normalise           153 non-null    float64
 9   one-percenters_away_team_normalise  153 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 12.1 KB


In [7]:
df.corr()

,kicks_home_team,one-percenters_home_team,kicks_away_team,one-percenters_away_team,home_win_lose,away_win_lose,kicks_home_team_normalise,one-percenters_home_team_normalise,kicks_away_team_normalise,one-percenters_away_team_normalise
kicks_home_team,1.000000,0.818009,-0.058689,-0.013512,0.396426,-0.397543,1.000000,0.818009,-0.058689,-0.013512
one-percenters_home_team,0.818009,1.000000,-0.081458,-0.058843,0.461639,-0.457679,0.818009,1.000000,-0.081458,-0.058843
kicks_away_team,-0.058689,-0.081458,1.000000,0.805290,-0.269043,0.263346,-0.058689,-0.081458,1.000000,0.805290
one-percenters_away_team,-0.013512,-0.058843,0.805290,1.000000,-0.242352,0.237538,-0.013512,-0.058843,0.805290,1.000000
home_win_lose,0.396426,0.461639,-0.269043,-0.242352,1.000000,-0.973795,0.396426,0.461639,-0.269043,-0.242352
away_win_lose,-0.397543,-0.457679,0.263346,0.237538,-0.973795,1.000000,-0.397543,-0.457679,0.263346,0.237538
kicks_home_team_normalise,1.000000,0.818009,-0.058689,-0.013512,0.396426,-0.397543,1.000000,0.818009,-0.058689,-0.013512
one-percenters_home_team_normalise,0.818009,1.000000,-0.081458,-0.058843,0.461639,-0.457679,0.818009,1.000000,-0.081458,-0.058843
kicks_away_team_normalise,-0.058689,-0.081458,1.000000,0.805290,-0.269043,0.263346,-0.058689,-0.081458,1.000000,0.805290
one-percenters_away_team_normalise,-0.013512,-0.058843,0.805290,1.000000,-0.242352,0.237538,-0.013512,-0.058843,0.805290,1.000000


In [8]:
def remove_zeroes_from_dataset(df, column):
    mask = df[column] != 0.0
    df = df.loc[mask]
    return df 

In [9]:
#df = remove_zeroes_from_dataset(df, 'bathrooms')

In [10]:
df.columns

Index(['kicks_home_team', 'one-percenters_home_team', 'kicks_away_team',
       'one-percenters_away_team', 'home_win_lose', 'away_win_lose',
       'kicks_home_team_normalise', 'one-percenters_home_team_normalise',
       'kicks_away_team_normalise', 'one-percenters_away_team_normalise'],
      dtype='object')

In [11]:
train, test = train_test_split(df[[df.columns[4], df.columns[5], df.columns[6], df.columns[7], df.columns[8], df.columns[9]]], test_size=0.2)

In [42]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self, df_rows_size, df_columns_size, output_size):
            self.x=torch.zeros(df_rows_size, df_columns_size)
            self.y=torch.zeros(df_rows_size, output_size)
            self.len= df_rows_size

    def __getitem__(self,index):

        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.len

In [45]:
# making dataset and filling the x and y values with the data tensors we need 
# rows: size of dataset set 
#columns: 4
dataset_train =Data(train.shape[0], 4, 2)
dataset_test =Data(test.shape[0], 4, 2)


In [46]:
train.columns

Index(['home_win_lose', 'away_win_lose', 'kicks_home_team_normalise',
       'one-percenters_home_team_normalise', 'kicks_away_team_normalise',
       'one-percenters_away_team_normalise'],
      dtype='object')

In [47]:
# making dataset and filling the x and y values with the data tensors we need 
# rows: size of dataset set 
#columns: 4
dataset_train =Data(train.shape[0], 4, 2)
dataset_test =Data(test.shape[0], 4, 2)

# convert dataframe columns to tensors 
x_feature_1 = torch.FloatTensor(train['kicks_home_team_normalise'].to_list())
x_feature_2 = torch.FloatTensor(train['one-percenters_home_team_normalise'].to_list())
x_feature_3 = torch.FloatTensor(train['kicks_away_team_normalise'].to_list())
x_feature_4 = torch.FloatTensor(train['one-percenters_away_team_normalise'].to_list())
y_feature_1 = torch.FloatTensor(train['home_win_lose'].to_list())
y_feature_2 = torch.FloatTensor(train['away_win_lose'].to_list())

# set tensor columns 
dataset_train.x[:, 0] = x_feature_1
dataset_train.x[:, 1] = x_feature_2
dataset_train.x[:, 2] = x_feature_3
dataset_train.x[:, 3] = x_feature_4
dataset_train.y[:, 0] = y_feature_1
dataset_train.y[:, 1] = y_feature_1

# convert dataframe columns to tensors 
x_feature_1 = torch.FloatTensor(test['kicks_home_team_normalise'].to_list())
x_feature_2 = torch.FloatTensor(test['one-percenters_home_team_normalise'].to_list())
x_feature_3 = torch.FloatTensor(test['kicks_away_team_normalise'].to_list())
x_feature_4 = torch.FloatTensor(test['one-percenters_away_team_normalise'].to_list())
y_feature_1 = torch.FloatTensor(test['home_win_lose'].to_list())
y_feature_2 = torch.FloatTensor(test['away_win_lose'].to_list())

# set tensor columns 
dataset_test.x[:, 0] = x_feature_1
dataset_test.x[:, 1] = x_feature_2
dataset_test.x[:, 2] = x_feature_3
dataset_test.x[:, 3] = x_feature_4
dataset_test.y[:, 0] = y_feature_1
dataset_test.y[:, 1] = y_feature_2

# create dataloader which loads the dataset for training 
# set batch size if batches are needed 
train_loader=DataLoader(dataset=dataset_train ,batch_size=150)
test_loader=DataLoader(dataset=dataset_train ,batch_size=150)

In [48]:
class logistic_regression(nn.Module):
    def __init__(self,input_size,output_size):
        super(logistic_regression,self).__init__()
        self.linear=nn.Linear(input_size,output_size)
        
    def forward(self,x):
        yhat=torch.sigmoid(self.linear(x))
        return yhat

In [49]:
# Create dataloader object, crierion function and optimizer.
model = logistic_regression(4, 2)
trainloader = DataLoader(dataset=dataset_train, batch_size=10)
#criterion_rms = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
learning_rate = .01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=.6)

In [50]:
def train_model(epochs, LOSS):
    for epoch in range(epochs):
        yhat = model(dataset_train.x)
        loss = criterion(yhat, dataset_train.y)
        LOSS.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return LOSS

In [38]:
LOSS = []
loss = train_model(500, LOSS)

RuntimeError: 1D target tensor expected, multi-target not supported